In [ ]:
# what's next? numpy

import numpy

A = numpy.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(A)

# OK, so what's so special about that compared to the list?

In [ ]:
# numpy arrays are fast, almost C speed
# as long as you do "large amounts of work"

import time

AL = range(0, 1000000)
BL = range(0, 1000000)
CL = [0] * len(AL)

start = time.time()
for i in range(0, len(AL)):
    CL[i] = AL[i] + BL[i]
print(time.time() - start)

A = numpy.array(range(0, 1000000), numpy.int32)
B = numpy.array(range(0, 1000000), numpy.int32)

start = time.time()
C = A + B
print(time.time() - start)


In [ ]:
# numpy notation is similar to array slicing
# and Matlab and Fortran matrix notation

A = numpy.array(range(0, 10))

V = A[::2] # this is a view (shallow copy)
V[0] = -10 # slices are views in numpy
print(V, A) 
B = A.copy() # this is a deep copy of A
B[0] = 0
print(B, A)

C = A[::2] + B[::2]
print(C)

C = A[1:9] * B[:8]
print(C)

C = A[1:-3] - B[2:-2]
print(C)

C = A / B[:5] # this is going to fail, because they aren't the same shape

In [ ]:
# numpy also supports multi-dimensional arrays
# default memory layout is:
# C, row-major, right-most index varies fastest

A = numpy.array(range(0, 8))
A = numpy.reshape(A, (2, 2, 2)) # change the shape of an array
                                # the total size (elements) must be the same
print(A)

print(A[0,0,0]) # this is different from nested lists
print(A[1,1,1])

A = numpy.transpose(A, axes=[0,2,1]) # swap around axes
print(A)

In [ ]:
# numpy also supports "broadcasting"

A = numpy.array(range(0, 4))
A = numpy.reshape(A, (2, 2))

print(A) # a 2x2 matrix

A = A + 1 # 1 is added to all elements
print(A)

v = numpy.array([-1, 1]) # let's make a vector
v = numpy.reshape(v, (2, 1)) # a column vector
print(v)

A = A * v # v gets broadcast over the columns
print(A)

v = numpy.reshape(v, (1, 2)) # now it's a row vector
print(v)

A = A - v # v gets broadcast over the rows
print(A)

In [ ]:
# you can use boolean arrays to filter out elements
A = numpy.array(range(1, 11))
b = numpy.array([i % 2 == 0 for i in range(1, 11)]) # all the even elements
print(b)

print(A[b]) # b is the same shape as A
            # this is stream compaction
            # the output size is equal to the number of Trues

print(numpy.where(b, A, 0)) # where generates the same shape as A
                            # but replaces A with 0 where b is False


In [ ]:
# numpy has a lot of functionality
# beyond +, *, - and /
# http://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs

A = numpy.array(range(0, 4))

print(numpy.max(A))
print(numpy.min(A))
print(numpy.sign(A))
print(numpy.cos(A))
print(A > A)
print(A == A)
print(-A)

In [ ]:
# and a lot of what you want is probably
# in the linear algebra
# http://docs.scipy.org/doc/numpy/reference/routines.linalg.html
    
from numpy.linalg import linalg # a submodule of a module

A = numpy.array([[0, 1], [2, 3]])
B = numpy.array([[0, -1], [1, 0]])

print(linalg.dot(A, B)) # matrix multiply
print(numpy.outer(A, B)) # outer product
print(linalg.qr(A)) # qr factorization
print(linalg.svd(A)) # SVD
print(linalg.eig(A)) # eigenvectors and values
print(linalg.inv(A)) # inverse of A
# etc. 

In [ ]:
# getting raw binary data in and out of numpy

A = numpy.arange(0, 10, .5, numpy.float32)
print(A)

f = open('foo.bin', 'wb')
A.tofile(f) # just do a to file and it will dump it in C-order
f.close()

la = len(A)
A = None
print(A)

f = open('foo.bin', 'rb')
A = numpy.fromfile(f, numpy.float32, la) # to read back in
                                         # you have to specify type and number
f.close()

print(A)

In [ ]:
# next is scipy
#
# it has lots of specialized functionality
# for scientific computing:
# FFTs, signal processing, integration, statistics,
# interpolation, optimization, graphs, etc.
#
# http://docs.scipy.org/doc/scipy/reference/

from scipy import fftpack

A = numpy.array([0, 1, 2, 3, 4, 3, 2, 1])

print(fftpack.fft(A)) # fft
print(fftpack.ifft(fftpack.fft(A))) # ifft and fft

from scipy import optimize

B = numpy.array([0, 1, 2, 3, 4, 5, 6, 7])

def poly(x, a, b, c): # the model to fit to
    return a + b*x + c*x*x

print(optimize.curve_fit(poly, B, A)) # outputs a, b, c and covariance matrix

In [ ]:
# the fun part, plotting the data

%matplotlib inline 
# this "magic" is necessary for ipython notebook
# it's not necessary (and will be an error)
# in normal python

import matplotlib.pyplot as plt # this is all you need in python
                                # pyplot is the Matlab like plotting interface
    
# examples of plots can be found at http://matplotlib.org/gallery.html

In [ ]:
import functools

A = numpy.array([0, 1, 2, 3, 4, 3, 2, 1])
B = numpy.array([0, 1, 2, 3, 4, 5, 6, 7])

def poly(x, a, b, c):
    return a + b*x + c*x*x

abc, cov = optimize.curve_fit(poly, B, A) # going to do the least squares fit like before

fixed = functools.partial(poly, a=abc[0], b=abc[1], c=abc[2]) # freeze the polynomial
fixed = numpy.vectorize(fixed) # create a vectorized version of the function

# the start of a plot
# pyplot is Matlab like, it is a state machine
plt.figure() # start a new plot
plt.xlabel('x') # labels
plt.ylabel('y')
plt.plot(B, fixed(B)) # the x and y values of the model
plt.legend('model')
plt.plot(B, A, 'o') # 'o' means plot it with circles
plt.legend('original')
plt.title('least squares fit to quadratic model') # a title
plt.show() # show it

# plt.savefig('foo.png') # write it to an image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#create a numpy array
a = np.random.normal(0,0.1,10000)
plt.hist(a,64,normed = 1, facecolor = 'green', alpha=0.5)
plt.show()
